Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [5]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#convert the created from an object to a date
df['created']=pd.to_datetime(df['created'])

In [7]:
df['created'].dtypes

dtype('<M8[ns]')

In [0]:
# more readable format
df['created']=df['created'].dt.strftime('%m/%d/%Y')

In [0]:
# back to date data type
df['created']=pd.to_datetime(df['created'])

###Engineer at least two new features

####Adding a column that shows if both cats and dogs are allowed.

In [10]:
# set condition

condition = (df['dogs_allowed']==1) & (df['cats_allowed']==1)
condition.value_counts()

False    27001
True     21816
dtype: int64

In [11]:
condition

0        False
1         True
2        False
3        False
4        False
         ...  
49347    False
49348     True
49349     True
49350     True
49351    False
Length: 48817, dtype: bool

In [12]:
# make the new column
newdf = pd.DataFrame(condition)
newdf = newdf.astype(int)
newdf.head()


,0
0,0
1,1
2,0
3,0
4,0


In [13]:
# add it to original df
df['dogs_and_cats_allowed'] = newdf
df.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,dogs_and_cats_allowed
0,1.5,3,2016-06-24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


####Making a column that shows the total number of rooms, both bed and bath.

In [16]:
df['total_rooms'] = df['bedrooms'] + df['bathrooms']
df.sample(3)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,dogs_and_cats_allowed,total_rooms
9174,1.0,1,2016-04-14,,6th Street,40.6719,-73.9865,2900,278 6th Street,low,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0
12904,1.0,1,2016-06-09,"Conveniently located near W4 subway station, w...",Bedford Street,40.7301,-74.0042,2925,39 Bedford Street,low,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0
6233,1.0,1,2016-04-14,Renovated 1 bedroom with separate kitchen incl...,W 47 Street,40.7608,-73.9891,2395,328 W 47 Street,low,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.0


####Make a column that shows the length of charaters used in the description

In [0]:
df['description_length'] = df['description'].str.len()


In [49]:
print(df['description_length'].isnull().sum())
df['description_length'] = df['description_length'].fillna(0)
print(df['description_length'].isnull().sum())

1425
0


###Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [50]:
df.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,dogs_and_cats_allowed,total_rooms,description_length
0,1.5,3,2016-06-24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,588.0


In [51]:
# split the data and check the shapes
train = df.query('created < 20160601')
test = df.query('created >= 20160601')
print(train.shape,test.shape)


(31844, 37) (16973, 37)


###Fit a linear regression model with at least two features.

In [0]:
# going to use the number of bedrooms and the total
#   number of rooms

In [0]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
# intaniate the class
model = LinearRegression()

In [57]:
#set the features
features = ['bedrooms',
            'total_rooms']
X_train = train[features]
X_test = test[features]
print(X_train.shape,X_test.shape)

(31844, 2) (16973, 2)


In [0]:
# set the target vectors
target = 'price'
y_train = train[target]
y_test = test[target]

In [59]:
# fit the model
model.fit(X_train,y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train,y_pred)
print(f'Train Error: {mae:.2f} percentage points')


Train Error: 818.53 percentage points


In [60]:
y_test_pred = model.predict(X_test)
mae_test = mean_absolute_error(y_test,y_test_pred)
print(f'Test Error: {mae_test:.2f} percentage points')

Test Error: 825.90 percentage points


###Get the model's coefficients and intercept.

In [61]:
print("Intercept:", model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept: 485.71869002321955
bedrooms      -1683.285220
total_rooms    2072.610116


###Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [64]:
#Train Data
mse = mean_squared_error(y_train,y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train,y_pred)
r2 = r2_score(y_train,y_pred)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1517879.6665142525
Root Mean Squared Error: 1232.0225917223486
Mean Absolute Error: 818.5310213271713
R^2: 0.5111543084316605


In [65]:
# Test Data
mse = mean_squared_error(y_test,y_test_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test,y_test_pred)
r2 = r2_score(y_test,y_test_pred)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1487715.3104108903
Root Mean Squared Error: 1219.719357233823
Mean Absolute Error: 825.8987822403526
R^2: 0.5213303957090346


###What's the best test MAE you can get? Share your score and features used with your cohort on Slack!

In [0]:
def test_train_mae(traindf,testdf,features):
  # intaniate the class
  model = LinearRegression()
  # set the features
  X_train = traindf[features]
  X_test = testdf[features]
  # set target
  target = 'price'
  y_train = traindf[target]
  y_test = testdf[target]
  # fit the data
  model.fit(X_train,y_train)
  # get the train error
  y_pred = model.predict(X_train)
  mae = mean_absolute_error(y_train,y_pred)
  print(f'Train Error: {mae:.2f} percentage points')
  # get the test error
  y_test_pred = model.predict(X_test)
  mae_test = mean_absolute_error(y_test,y_test_pred)
  print(f'Test Error: {mae_test:.2f} percentage points')  

  return mae_test

In [112]:
all_features = list(df.columns)
all_features.remove('interest_level')
all_features.remove('display_address')
all_features.remove('created')
all_features.remove('street_address')
all_features.remove('description')
all_features.remove('price')

all_features

['bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space',
 'dogs_and_cats_allowed',
 'total_rooms',
 'description_length']

In [120]:
best_test = 2000
for i in range(len(all_features)):
  for i2 in range(len(all_features)):
    if i < i2:
      features = [all_features[i],all_features[i2]]
      if test_train_mae(test,train,features) < best_test:
        best_test = test_train_mae(test,train,features)
        best_combo = [all_features[i],all_features[i2]]
      print(all_features[i],all_features[i2])
      print()

    else:
      continue

print(best_combo)

Train Error: 828.15 percentage points
Test Error: 821.53 percentage points
Train Error: 828.15 percentage points
Test Error: 821.53 percentage points
bathrooms bedrooms

Train Error: 889.60 percentage points
Test Error: 886.11 percentage points
bathrooms latitude

Train Error: 825.81 percentage points
Test Error: 825.08 percentage points
bathrooms longitude

Train Error: 870.79 percentage points
Test Error: 871.91 percentage points
bathrooms elevator

Train Error: 891.22 percentage points
Test Error: 888.73 percentage points
bathrooms cats_allowed

Train Error: 892.02 percentage points
Test Error: 890.01 percentage points
bathrooms hardwood_floors

Train Error: 890.24 percentage points
Test Error: 887.93 percentage points
bathrooms dogs_allowed

Train Error: 853.46 percentage points
Test Error: 859.19 percentage points
bathrooms doorman

Train Error: 876.40 percentage points
Test Error: 877.75 percentage points
bathrooms dishwasher

Train Error: 889.03 percentage points
Test Error: 887

In [121]:
# so it looks like the best test score is achieved with the longitude and the total rooms.
features = ['longitude','total_rooms']
test_train_mae(test,train,features)

Train Error: 819.81 percentage points
Test Error: 806.22 percentage points


806.2209918723362

###As always, commit your notebook to your fork of the GitHub repo.